In [ ]:
!pip install nltk==3.5
!pip install gensim
!pip install umap-learn
!pip install wikipedia
!pip install unidecode

In [ ]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import wikipedia
import string
from unidecode import unidecode
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import urllib.request
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')
nltk.download('punkt')

import matplotlib.pyplot as plt
%matplotlib inline

# Definição do Corpus

## Base

In [ ]:
wikipedia.set_lang("pt")
bh = wikipedia.page("Belo Horizonte")

In [ ]:
corpus = bh.content

In [ ]:
print(corpus)

Selecionamos algumas frases do corpus de BH da wikipedia.

Conside a lista abaixo como nosso corpus de documentos. Cada elemento da lista, considere como um único documento.

In [ ]:
documentos = \
["Belo Horizonte é um município brasileiro e a capital do estado de Minas Gerais",
"A populacao de Belo Horizonte é estimada em 2 501 576 habitantes, conforme estimativas do Instituto Brasileiro de Geografia e Estatística",
"Belo Horizonte já foi indicada pelo Population Crisis Commitee, da ONU, como a metrópole com melhor qualidade de vida na América Latina",
"Belo Horizonte é mundialmente conhecida e exerce significativa influência nacional e até internacional, seja do ponto de vista cultural, econômico ou político",
"Belo Horizonte é a capital do segundo estado mais populoso do Brasil, Minas Gerais"]

## Preprocessamento

<b> Atividade </b>

1) Escreva uma método que realiza o pré-processamento da lista de <b>documentos</b>.

O método deve, para cada documento:
- tokenizar cada palavra
- remover stopwords
- remover números
- remover pontuções
- remover acentos

In [ ]:
from nltk.corpus import stopwords
nltk.download("stopwords")

# Representação Textual

## TD-IDF

Dica de leitura: https://kavita-ganesan.com/tfidftransformer-tfidfvectorizer-usage-differences/#.XklQxnVKj7c

Para representar o texto com TF-IDF utilizamos o TfidfVectorizer. A seguir apresentamos instruções sobre como utilizá-lo.

```python
#primeiro criamos o objeto
vect = TfidfVectorizer()
vect #aqui você pode observa todos os parâmetros que o objeto possui
## Existem alguns parâmetros, opcionais, que podemos informar para uma possível melhora do nosso modelo. Por exemplo:
### inclui 1-grams e 2-grams
vect.set_params(ngram_range=(1, 2))
### ignora termos que a aparecem em mais de 50% dos documentoss
vect.set_params(max_df=0.5)
### só considero termos que aparecem em ao menos 2 documentos
vect.set_params(min_df=2)

#depois aplicamos fit_transform para transformar o texto em números
docs_tdidf = vect.fit_transform(docs)

#o docs_tdidf é uma matriz com os números que representam cada um dos textos.
## Conseguimos verificar a dimensão desta matriz:
print(docs_tdidf.shape)

#Para visualizar as features capturadas pelo TF-IDF utilize:
print(vect.get_feature_names_out())
#Para visualizar os vetores correspondentes a cada palavara utilize:
df = pd.DataFrame(docs_tdidf.T.todense(), index=vect.get_feature_names_out(), columns=["doc"+str(i+1) for i in range(0,len(docs))])
df
```

<b> Atividade: </b>

2) Faça o TDIFTVectorizer nos documentos da variável <b>documentos</b> sem alterar nenhum parâmetro.

<b> Atividade </b>

3) Imprima o shape do resultado da atividade 2

<b> Atividade </b>

4) Imprima as features capturadas em 2.

5) Imprima os vetores correspondentes a cada palavra de cada documento.

## Bag of Words

Para representar o bag of words utilizamos o CountVectorizer

```python
#primeiro criamos o objeto
vect_bag = CountVectorizer(binary=False) #se binary = False -> ocorre a contagem da frequência em que a palavra aparece
vect_bag #imprime os parâmetros

```

<b> Atividade </b>

6) Faça o CountVectorizer nos documentos da variável <b>documentos</b> considerando binary = True

<b> Atividade </b>

7) Imprima o índice correspondente a cada token da lista retornada por vect_bag.get_feature_names_out()

## Embedding

### Treinando seu embedding

Aqui vamos utilizar o corpus machado. São textos/contos escritos por Machado de Assis.
Esse corpus é diponibilizado pelo NLTK.

In [ ]:
from nltk.corpus import machado

In [ ]:
import nltk
from nltk.corpus import machado
nltk.download('machado')

In [ ]:
raw_casmurro = machado.raw('contos/macn001.txt')

O método ''machado_sents()'' retorna todo o texto quebrado pelas setenças e já tokenizado.

As sentenças são separadas pelo "\n". Dentro de cada sentença, divide os tokens separadas pelo espaço.

In [ ]:
machado_sents = machado.sents()
print(machado_sents)

In [ ]:
len(machado_sents)

Vamos relizar um pré-processamento mínimo nos dados. Lembrando que: o pré-processamento é impotatíssimo no resultado final.

<b> Atividade </b>

8) Aplique as técnicas abaixo no documento <b> machado_sents</b>:

- lower
- remoção pontuações

<b> Treinando o embedding </b>

Para treinar os embeddings existem alguns parâmetros, vide exemplo abaixo:

```python
#Alguns parâmetros:
## size -> dimensão vetor
## min_count -> ignora todas palavras cuja frequência mínima é menor que este
## workers -> quantas threads serão utilizadas para treinar o modelo
## seed -> seed para geração do numero aleatório.
## sg -> 1 para skip-gram; caso contrário CBOW.
## window -> contexto, Distância máxima entre a palavra atual e a prevista em uma frase. O default é 5.
model = word2vec.Word2Vec(text_preproc, min_count=10, workers=4, seed=123, sg=1, vector_size=300, window=5)
```

<b> Atividade </b>

9) Gere os embeddings com o texto processado do documento de Machado de Assis.



<b> Atividade </b>

10) Faça os itens abaixo:

- Verifique o vetor de embeddings da variável "dom"
- Verifique a similaridade entre "mulher" e "homem"
- Verifique a similaridade entre "dom" e "casmurro"

<b> Atividade </b>

Dada as seguintes palavras:

foi, relógio, amor, raiva, brasil.

11) Escreva um método que retorne uma lista com as 5 palavras similares de cada uma das listadas anteriormente.
Imprima a lista das palavras similares, incluindo a palavra origem.

### Visualização

Para a visualização dos embeddings iremos  construir um array com todas as palavras retornadas anteriormente.

<b> Atividade </b>

12) Primeiro, gere uma única lista com todas as palavras retornadas anteriomente. O array deve ter tamanho 30.

13) Gere um array com todos os embeddings das palavras anteriores. Este terá dimensão (30,300)

<b>Dica: </b> Use a função abaixo para plotar o array 2D que será gerado com o método PCA e TSNE

In [ ]:
def plot_embedding_2d(array_2d, all_words, words_seed):
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    for (x, y), w in zip(array_2d, all_words):
        ax.scatter(x, y, c='red' if w in words_seed else 'blue')
        ax.annotate(w,
                     xy=(x, y),
                     xytext=(5, 2),
                     textcoords='offset points',
                     ha='right',
                     va='bottom')

#### PCA

<b> É uma ténica que existe a mais de século. É rápido, determinístico e linear. Essa linearidade limita sua utilidade em domínios complexos, como linguagem natural ou imagens, onde a estrutura não linear. </b>

Mais informações: https://medium.com/towards-artificial-intelligence/machine-learning-dimensionality-reduction-via-principal-component-analysis-1bdc77462831


<b> Atividade </b>

14) Gere a visualização dos embeddings anteriores utilizando o PCA para reduzir a dimensionalidade.

Exemplo do PCA:

```python
#uso de PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(array_embeddings)
```

#### TSNE
<b> Uma técnica mais recente que captura estrutura não linear é o t-SNE, que significa distribuição estocástica de embedding viziznhos em t ( t-distributed Stochastic Neighbor Embedding).
É uma ótima técnica para capturar a estrutura não linear em dados de alta dimensão(pelo menos em nível local). Isto é, dois pontos que são próximos no espaço de alta dimensão a probabilidade de estarem próximos em uma dimensão baixa é alta. </b>

Mais informações: https://medium.com/@garora039/dimensionality-reduction-using-t-sne-effectively-cabb2cd519b

<b> Atividade </b>

15) Gere a visualização dos embeddings anteriores utilizando o TSNE para reduzir a dimensionalidade.

Exemplo do TSNE:

```python
#uso de TSNE
tsne = TSNE(n_components=2, random_state=0)
tsne_result =  tsne.fit_transform(array_embeddings)
```

# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [ ]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado"
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Similaridade de Jaccard

Divisão do número de tokens em comuns nas duas frases pelo tamanho do vocabulario das duas frases.

<b> Atividade </b>

16) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

## Distância e similaridade cosseno

Aqui iremos calcular a distância e a similaridade utilizando embedding

- Embedding

In [ ]:
def embedding_da_frase(tokens_frase):
  return np.mean(np.array([word_vectors[token] for token in tokens_frase if token in word_vectors.key_to_index]), axis=0)

In [ ]:
frase1_emb = embedding_da_frase(frase1_pre)
frase2_emb = embedding_da_frase(frase2_pre)

In [ ]:
distance.cosine(frase1_emb, frase2_emb)

In [ ]:
cosine_similarity(frase1_emb.reshape((1, 100)), frase2_emb.reshape((1, 100)))

In [ ]:
distance.euclidean(frase1_emb, frase2_emb)